<a href="https://colab.research.google.com/github/CarlosMejia07/Proyecto-Kaggle-UdeA-20252/blob/main/05%20-%20third_attempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importe, descompresión e inspección de los datos**

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia
!unzip udea*.zip > /dev/null
!wc *.csv

udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)
replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
   296787    296787   4716673 submission_example.csv
   296787   4565553  59185238 test.csv
   692501  10666231 143732437 train.csv
  1286075  15528571 207634348 total


**Importe de las librerias a usar y lectura del archivo train**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install category_encoders

z = pd.read_csv("train.csv")
print ("Estructura de los datos cargados", z.shape)
z_clean = z.copy()

Estructura de los datos cargados (692500, 21)


**Preprocesado de train**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder
from lightgbm import LGBMClassifier

z_clean = z_clean.drop(columns=['ID', 'PERIODO_ACADEMICO'])

non_numeric_cols = z_clean.select_dtypes(exclude=np.number).columns

for col in non_numeric_cols:
    z_clean[col] = z_clean[col].fillna(z_clean[col].mode()[0])

z_clean.isna().sum()

def replace_binary(col):
    return col.replace({
        'Si': 1, 'Sí': 1, 'S': 1, 'Y': 1, '1': 1, 'si': 1, 'sí': 1,
        'No': 0, 'N': 0, '0': 0, 'no': 0
    })

binary_cols = ['F_TIENEINTERNET', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL',
               'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR',
               'F_TIENEINTERNET.1']

for col in binary_cols:
    z_clean[col] = replace_binary(z_clean[col])

z_clean[binary_cols] = z_clean[binary_cols].astype(int)

map_estrato = {
    'Sin Estrato': 0,
    'Estrato 1': 1,
    'Estrato 2': 2,
    'Estrato 3': 3,
    'Estrato 4': 4,
    'Estrato 5': 5,
    'Estrato 6': 6

}

map_matricula = {
    "No pagó matrícula": 0,
    "Menos de 500 mil": 1,
    "Entre 500 mil y menos de 1 millón": 2,
    "Entre 1 millón y menos de 2.5 millones": 3,
    "Entre 2.5 millones y menos de 4 millones": 4,
    "Entre 4 millones y menos de 5.5 millones": 5,
    "Entre 5.5 millones y menos de 7 millones": 6,
    "Más de 7 millones": 7
}

map_horas = {
    "0": 0,
    "Menos de 10 horas": 1,
    "Entre 11 y 20 horas": 2,
    "Entre 21 y 30 horas": 3,
    "Más de 30 horas": 4
}

z_clean['F_EDUCACIONPADRE'] = z_clean['F_EDUCACIONPADRE'].replace(['No sabe', 'No Aplica'], 'Desconocido')
z_clean['F_EDUCACIONMADRE'] = z_clean['F_EDUCACIONMADRE'].replace(['No sabe', 'No Aplica'], 'Desconocido')

map_educacion = {
    "Primaria incompleta": 1,
    "Primaria completa": 2,
    "Secundaria (Bachillerato) incompleta": 3,
    "Secundaria (Bachillerato) completa": 4,
    "Técnica o tecnológica incompleta": 5,
    "Técnica o tecnológica completa": 6,
    "Educación profesional incompleta": 7,
    "Educación profesional completa": 8,
    "Postgrado": 9,
    "Desconocido": 0
}

z_clean["F_ESTRATOVIVIENDA"] = z_clean["F_ESTRATOVIVIENDA"].map(map_estrato)
z_clean["E_VALORMATRICULAUNIVERSIDAD"] = z_clean["E_VALORMATRICULAUNIVERSIDAD"].map(map_matricula)
z_clean["E_HORASSEMANATRABAJA"] = z_clean["E_HORASSEMANATRABAJA"].map(map_horas)
z_clean["F_EDUCACIONPADRE"] = z_clean["F_EDUCACIONPADRE"].map(map_educacion)
z_clean["F_EDUCACIONMADRE"] = z_clean["F_EDUCACIONMADRE"].map(map_educacion)

nominal_cols = [
    "E_PRGM_ACADEMICO",
    "E_PRGM_DEPARTAMENTO"
]

te = TargetEncoder(cols=nominal_cols)
z_clean[nominal_cols] = te.fit_transform(z_clean[nominal_cols], z_clean["RENDIMIENTO_GLOBAL"])

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
cols_numericas = ['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']

z_clean[cols_numericas] = scaler.fit_transform(z_clean[cols_numericas])

#Variable objetivo
# Definimos el orden lógico
orden = ['bajo', 'medio-bajo', 'medio-alto', 'alto']

z_clean['RENDIMIENTO_GLOBAL'] = pd.Categorical(
    z_clean['RENDIMIENTO_GLOBAL'],
    categories=orden,
    ordered=True
).codes

/tmp/ipython-input-2111746733.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return col.replace({


***Modelado***

In [ ]:
X = z_clean.drop(columns=["RENDIMIENTO_GLOBAL"])
y = z_clean["RENDIMIENTO_GLOBAL"]

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = LGBMClassifier(
    n_estimators=5000,
    learning_rate=0.03,
    num_leaves=31,
    max_depth=-1,
    objective="multiclass",
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val)
)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.170764 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1128
[LightGBM] [Info] Number of data points in the train set: 692500, number of used features: 18
[LightGBM] [Info] Start training from score -1.387092
[LightGBM] [Info] Start training from score -1.391216
[LightGBM] [Info] Start training from score -1.395031
[LightGBM] [Info] Start training from score -1.371991


LGBMClassifier(colsample_bytree=0.8, learning_rate=0.03, n_estimators=5000,
               objective='multiclass', random_state=42, subsample=0.8)

In [ ]:
model.fit(X, y)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.139844 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1126
[LightGBM] [Info] Number of data points in the train set: 692500, number of used features: 18
[LightGBM] [Info] Start training from score -1.387092
[LightGBM] [Info] Start training from score -1.391216
[LightGBM] [Info] Start training from score -1.395031
[LightGBM] [Info] Start training from score -1.371991


LGBMClassifier(colsample_bytree=0.9, learning_rate=0.03, n_estimators=1000,
               subsample=0.8)

**Carga y preprocesado de los datos del archivo test**

In [ ]:
test = pd.read_csv("test.csv")
test_ids = test["ID"]  # Guardamos los IDs como referencia
test_clean = test.copy()

test_clean = test_clean.drop(columns=['ID', 'PERIODO_ACADEMICO'])

non_numeric_cols = test_clean.select_dtypes(exclude=np.number).columns

for col in non_numeric_cols:
    test_clean[col] = test_clean[col].fillna(test_clean[col].mode()[0])

test_clean.isna().sum()

def replace_binary(col):
    return col.replace({
        'Si': 1, 'Sí': 1, 'S': 1, 'Y': 1, '1': 1, 'si': 1, 'sí': 1,
        'No': 0, 'N': 0, '0': 0, 'no': 0
    })

binary_cols = ['F_TIENEINTERNET', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL',
               'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR',
               'F_TIENEINTERNET.1']

for col in binary_cols:
    test_clean[col] = replace_binary(test_clean[col])

test_clean[binary_cols] = test_clean[binary_cols].astype(int)

map_estrato = {
    'Sin Estrato': 0,
    'Estrato 1': 1,
    'Estrato 2': 2,
    'Estrato 3': 3,
    'Estrato 4': 4,
    'Estrato 5': 5,
    'Estrato 6': 6

}

map_matricula = {
    "No pagó matrícula": 0,
    "Menos de 500 mil": 1,
    "Entre 500 mil y menos de 1 millón": 2,
    "Entre 1 millón y menos de 2.5 millones": 3,
    "Entre 2.5 millones y menos de 4 millones": 4,
    "Entre 4 millones y menos de 5.5 millones": 5,
    "Entre 5.5 millones y menos de 7 millones": 6,
    "Más de 7 millones": 7
}

map_horas = {
    "0": 0,
    "Menos de 10 horas": 1,
    "Entre 11 y 20 horas": 2,
    "Entre 21 y 30 horas": 3,
    "Más de 30 horas": 4
}

test_clean['F_EDUCACIONPADRE'] = test_clean['F_EDUCACIONPADRE'].replace(['No sabe', 'No Aplica'], 'Desconocido')
test_clean['F_EDUCACIONMADRE'] = test_clean['F_EDUCACIONMADRE'].replace(['No sabe', 'No Aplica'], 'Desconocido')

map_educacion = {
    "Primaria incompleta": 1,
    "Primaria completa": 2,
    "Secundaria (Bachillerato) incompleta": 3,
    "Secundaria (Bachillerato) completa": 4,
    "Técnica o tecnológica incompleta": 5,
    "Técnica o tecnológica completa": 6,
    "Educación profesional incompleta": 7,
    "Educación profesional completa": 8,
    "Postgrado": 9,
    "Desconocido": 0
}

test_clean["F_ESTRATOVIVIENDA"] = test_clean["F_ESTRATOVIVIENDA"].map(map_estrato)
test_clean["E_VALORMATRICULAUNIVERSIDAD"] = test_clean["E_VALORMATRICULAUNIVERSIDAD"].map(map_matricula)
test_clean["E_HORASSEMANATRABAJA"] = test_clean["E_HORASSEMANATRABAJA"].map(map_horas)
test_clean["F_EDUCACIONPADRE"] = test_clean["F_EDUCACIONPADRE"].map(map_educacion)
test_clean["F_EDUCACIONMADRE"] = test_clean["F_EDUCACIONMADRE"].map(map_educacion)

nominal_cols = [
    "E_PRGM_ACADEMICO",
    "E_PRGM_DEPARTAMENTO"
]

test_clean[nominal_cols] = te.transform(test_clean[nominal_cols])

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
cols_numericas = ['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']

test_clean[cols_numericas] = scaler.fit_transform(test_clean[cols_numericas])

/tmp/ipython-input-172175962.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return col.replace({


**Predicciones**

In [ ]:
test_predictions = model.predict(test_clean)

**Preparando el archivo solución**

In [ ]:
map_dict = {0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto', 3: 'alto'}
test_predictions_cat = [map_dict[i] for i in test_predictions]

solucion = pd.DataFrame({
    'ID': test_ids,
    'RENDIMIENTO_GLOBAL': test_predictions_cat
})
solucion.to_csv('submission4.csv', index=False)